In [1]:
from dataset.compas import CompasDataset
from dataset import dataset_loader
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
name = 'compas'
dataset_ares = dataset_loader(name, data_path='rawdata/', dropped_features=[], n_bins=None)

dataset = CompasDataset(dataset_ares=dataset_ares)
df = dataset.get_dataframe()

In [4]:
df.head()

,Sex = Female,Sex = Male,Age_Cat = Less than 25,Age_Cat = 25 - 45,Age_Cat = Greater than 45,Race = African-American,Race = Asian,Race = Caucasian,Race = Hispanic,Race = Native American,Race = Other,C_Charge_Degree = F,C_Charge_Degree = M,Priors_Count,Time_Served,Status
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
1,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,10.0,0.0
2,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,1.0,0.0
3,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
4,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,14.0,6.0,0.0


### 1.standardization

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

##### use the standardscaler

In [6]:
# scaler = StandardScaler()

# # Assume df is your DataFrame and 'target' is your target column
# X = df.drop(columns=['Status'])
# Y = df['Status']

# # Step 1: Split the Data
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# X_train_scaler = scaler.fit_transform(X_train)
# X_test_scaler = scaler.transform(X_test)

# print(f'X_train_scaler: {X_train_scaler.shape}')
# print(f'X_test_scaler: {X_test_scaler.shape}')

##### don't use the standardscaler

In [7]:
# Assume df is your DataFrame and 'target' is your target column
X = df.drop(columns=['Status'])
Y = df['Status']

# Step 1: Split the Data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

X_train_scaler = X_train.values
X_test_scaler = X_test.values

print(f'X_train_scaler: {X_train_scaler.shape}')
print(f'X_test_scaler: {X_test_scaler.shape}')

X_train_scaler: (4937, 15)
X_test_scaler: (1235, 15)


In [8]:
from models import PyTorchDNN, PyTorchLinearSVM, PyTorchRBFNet,PyTorchLogisticRegression

In [9]:
input_dim = X_train_scaler.shape[1]

model_svm = PyTorchLinearSVM(input_dim)

# 定义损失函数和优化器
criterion = nn.BCEWithLogitsLoss()  # 二分类任务中的损失函数
optimizer = optim.Adam(model_svm.parameters(), lr=0.01)  # 使用Adam优化器

# 数据转换为 PyTorch tensor 格式
X_train_tensor = torch.FloatTensor(X_train_scaler)
y_train_tensor = torch.FloatTensor(y_train).view(-1, 1)  # 确保y是二维



# 训练模型
epochs = 400  # 训练次数
for epoch in range(epochs):
    model_svm.train()  # 进入训练模式
    # 前向传播
    outputs = model_svm(X_train_tensor)
    # 计算损失
    loss = criterion(outputs, y_train_tensor)
    # 梯度清零
    optimizer.zero_grad()
    # 反向传播
    loss.backward()
    # 优化更新参数
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [10/400], Loss: 0.6925
Epoch [20/400], Loss: 0.6864
Epoch [30/400], Loss: 0.6468
Epoch [40/400], Loss: 0.6257
Epoch [50/400], Loss: 0.6195
Epoch [60/400], Loss: 0.6143
Epoch [70/400], Loss: 0.6122
Epoch [80/400], Loss: 0.6107
Epoch [90/400], Loss: 0.6099
Epoch [100/400], Loss: 0.6093
Epoch [110/400], Loss: 0.6089
Epoch [120/400], Loss: 0.6086
Epoch [130/400], Loss: 0.6084
Epoch [140/400], Loss: 0.6082
Epoch [150/400], Loss: 0.6081
Epoch [160/400], Loss: 0.6081
Epoch [170/400], Loss: 0.6080
Epoch [180/400], Loss: 0.6080
Epoch [190/400], Loss: 0.6080
Epoch [200/400], Loss: 0.6080
Epoch [210/400], Loss: 0.6080
Epoch [220/400], Loss: 0.6080
Epoch [230/400], Loss: 0.6080
Epoch [240/400], Loss: 0.6080
Epoch [250/400], Loss: 0.6080
Epoch [260/400], Loss: 0.6080
Epoch [270/400], Loss: 0.6080
Epoch [280/400], Loss: 0.6080
Epoch [290/400], Loss: 0.6080
Epoch [300/400], Loss: 0.6080
Epoch [310/400], Loss: 0.6080
Epoch [320/400], Loss: 0.6080
Epoch [330/400], Loss: 0.6080
Epoch [340/400], Lo

### 2. initialize MODEL and DATA

In [10]:
from xai_cola.ml_model import Model
from xai_cola import data

In [11]:
column_names = X.columns.tolist()
column_names

['Sex = Female',
 'Sex = Male',
 'Age_Cat = Less than 25',
 'Age_Cat = 25 - 45',
 'Age_Cat = Greater than 45',
 'Race = African-American',
 'Race = Asian',
 'Race = Caucasian',
 'Race = Hispanic',
 'Race = Native American',
 'Race = Other',
 'C_Charge_Degree = F',
 'C_Charge_Degree = M',
 'Priors_Count',
 'Time_Served']

In [12]:
ml_model = Model(model_svm,'PYT')
compas_ares = data.NumpyData(
    data=X_test_scaler, 
    target_name="Status", 
    feature_names=column_names,
    )


##### Test counterfactual_explainer/ares.py

In [15]:
from xai_cola.counterfactual_explainer.ares import ARecourseS
ares_test = ARecourseS(
    ml_model=ml_model,
)

In [16]:
a,b = ares_test.generate_counterfactuals(
    dataset_ares=dataset_ares,
    data=compas_ares,
)

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


Computing Candidate Sets of Conjunctions of Predicates SD and RL
Processing 72 combinations | Sampling itemset size 43
SD and RL Computed with Lengths 69 and 69
Computing Ground Set of Triples V
Reducing RL
RL Reduced from Size 69 to 47


100%|██████████| 69/69 [00:00<00:00, 8624.60it/s]


Ground Set Computed with Length 1104


0it [00:00, ?it/s]np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more infor

...

100%|██████████| 199/199 [00:00<00:00, 19869.70it/s]


Checking Exchange


100%|██████████| 188/188 [00:00<00:00, 2912.16it/s]


Checking Delete
Checking Add


100%|██████████| 188/188 [00:00<00:00, 93827.84it/s]
0it [00:00, ?it/s]np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/

In [17]:
a

array([[ 0.,  1.,  0., ...,  0.,  7., 19.],
       [ 0.,  1.,  0., ...,  1.,  8., 62.],
       [ 0.,  1.,  0., ...,  1.,  0.,  0.],
       ...,
       [ 0.,  1.,  1., ...,  1.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  1.],
       [ 1.,  0.,  0., ...,  0.,  3.,  1.]])

In [18]:
b

array([[ 0.,  1.,  1., ...,  0.,  1., 33.],
       [ 0.,  1.,  1., ...,  1.,  0.,  0.],
       [ 0.,  1.,  1., ...,  1.,  0.,  0.],
       ...,
       [ 0.,  1.,  0., ...,  0.,  1.,  1.],
       [ 1.,  0.,  0., ...,  0.,  7.,  0.],
       [ 1.,  0.,  1., ...,  0.,  1.,  2.]])

##### Test the DisCount

In [19]:
from xai_cola.counterfactual_explainer import DisCount

discount_test = DisCount(
    ml_model = ml_model,
)

In [23]:
compas_ares.get_x()

,Sex = Female,Sex = Male,Age_Cat = Less than 25,Age_Cat = 25 - 45,Age_Cat = Greater than 45,Race = African-American,Race = Asian,Race = Caucasian,Race = Hispanic,Race = Native American,Race = Other,C_Charge_Degree = F,C_Charge_Degree = M,Priors_Count,Time_Served
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,13.0,7.0
1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0
2,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0
3,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,42.0
1231,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.0,1.0
1232,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,6.0,0.0
1233,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [26]:
X_test_scaler_screwed = X_test.head(100).values

In [27]:
compas_ares_screwed = data.NumpyData(
    data=X_test_scaler_screwed, 
    target_name="Status", 
    feature_names=column_names,
    )

In [30]:
a,b = discount_test.generate_counterfactuals(
    data=compas_ares_screwed,
    lr=1e-1,
    n_proj=10,
    delta=0.05,
    U_1=0.4,
    U_2=0.3,
    l=0.2,
    r=1,
    max_iter=15,
    tau=1e1,   #差值变化 - 步长不能太大或者太小
    silent=False,
    )

INFO:root:Optimization started
  0%|          | 0/15 [00:00<?, ?it/s]INFO:root:U_1-Qu_upper=0.4, U_2-Qv_upper=-0.547748463532679
INFO:root:eta=1, l=0.24000000000000002, r=1
INFO:root:Iter 1: Q = 0.47103461623191833, term1 = 0.028260568156838417, term2 = 0.47103461623191833
  7%|▋         | 1/15 [00:09<02:10,  9.30s/it]INFO:root:U_1-Qu_upper=0.4, U_2-Qv_upper=-0.5219312422875813
INFO:root:eta=1, l=0.278, r=1
INFO:root:Iter 2: Q = 0.44649380445480347, term1 = 0.02826366387307644, term2 = 0.44649380445480347
 13%|█▎        | 2/15 [00:18<01:59,  9.19s/it]INFO:root:U_1-Qu_upper=0.2894437400130965, U_2-Qv_upper=-0.4858123258710198
INFO:root:eta=1, l=0.31410000000000005, r=1
INFO:root:Iter 3: Q = 0.42323169112205505, term1 = 0.028270795941352844, term2 = 0.42323169112205505
 20%|██        | 3/15 [00:27<01:50,  9.17s/it]INFO:root:U_1-Qu_upper=0.4, U_2-Qv_upper=-0.46088797086928085
INFO:root:eta=1, l=0.34839500000000007, r=1
INFO:root:Iter 4: Q = 0.4011814296245575, term1 = 0.028279060497879982

In [33]:
a

array([[ 0.,  1.,  0., ...,  1., 13.,  7.],
       [ 0.,  1.,  0., ...,  1.,  1.,  1.],
       [ 0.,  1.,  0., ...,  0.,  2.,  0.],
       ...,
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  5., 54.],
       [ 0.,  1.,  0., ...,  1.,  0.,  0.]])

In [34]:
b

array([[ 2.99187284e-02,  1.01217365e+00, -2.16390360e-02, ...,
         6.04006229e-03,  4.97781897e+00,  2.41981308e+02],
       [-8.82729888e-03,  1.00820661e+00,  1.80105083e-02, ...,
        -4.84061660e-03,  2.00410151e+00,  2.98757434e+00],
       [ 2.28304863e-02,  1.01778173e+00,  9.73067820e-01, ...,
         1.36264004e-02, -3.12075811e-03,  1.00157452e+00],
       ...,
       [-4.93710972e-02,  9.83776033e-01,  3.96627598e-02, ...,
        -2.04031225e-02,  1.82100218e-02,  6.73706364e-03],
       [-8.27650577e-02,  9.78479922e-01,  1.17631972e-01, ...,
        -7.36575574e-02,  1.04073918e+00,  2.99291778e+00],
       [ 5.59007004e-02,  1.01803267e+00,  9.41679239e-01, ...,
         1.24509735e-02,  1.97904360e+00,  1.29943886e+01]], dtype=float32)

##### DisCOUNT test on scalered Compas dataset

In [51]:
scaler = StandardScaler()

# Assume df is your DataFrame and 'target' is your target column
X = df.drop(columns=['Status'])
Y = df['Status']

# Step 1: Split the Data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

X_train_scaler = scaler.fit_transform(X_train)
X_test_scaler = scaler.transform(X_test)

print(f'X_train_scaler: {X_train_scaler.shape}')
print(f'X_test_scaler: {X_test_scaler.shape}')

X_train_scaler: (4937, 15)
X_test_scaler: (1235, 15)


In [52]:
input_dim = X_train_scaler.shape[1]

model_svm_scaler = PyTorchLinearSVM(input_dim)

# 定义损失函数和优化器
criterion = nn.BCEWithLogitsLoss()  # 二分类任务中的损失函数
optimizer = optim.Adam(model_svm.parameters(), lr=0.01)  # 使用Adam优化器

# 数据转换为 PyTorch tensor 格式
X_train_tensor = torch.FloatTensor(X_train_scaler)
y_train_tensor = torch.FloatTensor(y_train).view(-1, 1)  # 确保y是二维



# 训练模型
epochs = 400  # 训练次数
for epoch in range(epochs):
    model_svm.train()  # 进入训练模式
    # 前向传播
    outputs = model_svm(X_train_tensor)
    # 计算损失
    loss = criterion(outputs, y_train_tensor)
    # 梯度清零
    optimizer.zero_grad()
    # 反向传播
    loss.backward()
    # 优化更新参数
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [10/400], Loss: 0.6967
Epoch [20/400], Loss: 0.6586
Epoch [30/400], Loss: 0.6362
Epoch [40/400], Loss: 0.6233
Epoch [50/400], Loss: 0.6158
Epoch [60/400], Loss: 0.6116
Epoch [70/400], Loss: 0.6097
Epoch [80/400], Loss: 0.6087
Epoch [90/400], Loss: 0.6084
Epoch [100/400], Loss: 0.6083
Epoch [110/400], Loss: 0.6081
Epoch [120/400], Loss: 0.6081
Epoch [130/400], Loss: 0.6080
Epoch [140/400], Loss: 0.6080
Epoch [150/400], Loss: 0.6080
Epoch [160/400], Loss: 0.6080
Epoch [170/400], Loss: 0.6080
Epoch [180/400], Loss: 0.6080
Epoch [190/400], Loss: 0.6080
Epoch [200/400], Loss: 0.6080
Epoch [210/400], Loss: 0.6080
Epoch [220/400], Loss: 0.6080
Epoch [230/400], Loss: 0.6080
Epoch [240/400], Loss: 0.6080
Epoch [250/400], Loss: 0.6080
Epoch [260/400], Loss: 0.6080
Epoch [270/400], Loss: 0.6080
Epoch [280/400], Loss: 0.6080
Epoch [290/400], Loss: 0.6080
Epoch [300/400], Loss: 0.6080
Epoch [310/400], Loss: 0.6080
Epoch [320/400], Loss: 0.6080
Epoch [330/400], Loss: 0.6080
Epoch [340/400], Lo

In [53]:
ml_model_scaler = Model(model_svm_scaler,'PYT')
compas_ares_scaler_test = data.NumpyData(
    data=X_test_scaler, 
    target_name="Status", 
    feature_names=column_names,
    )

In [54]:
discount_test_scaler = DisCount(ml_model = ml_model_scaler)

INFO:root:Optimization started
  0%|          | 0/15 [00:00<?, ?it/s]INFO:root:U_1-Qu_upper=0.38395383913330317, U_2-Qv_upper=-0.3228477135793834
INFO:root:eta=1, l=0.24000000000000002, r=1
INFO:root:Iter 1: Q = 25.21678924560547, term1 = 8.865663528442383, term2 = 25.21678924560547
  7%|▋         | 1/15 [00:00<00:13,  1.03it/s]INFO:root:U_1-Qu_upper=-3.2804367389230213, U_2-Qv_upper=-4.638451458001744
INFO:root:eta=0.6851664198833262, l=0.278, r=1
INFO:root:Iter 2: Q = 4624.89306640625, term1 = 4592.8125, term2 = 4639.63427734375
 13%|█▎        | 2/15 [00:01<00:13,  1.00s/it]INFO:root:U_1-Qu_upper=-13.53455126683212, U_2-Qv_upper=-6.13176755018246
INFO:root:eta=0.5031126005036357, l=0.278, r=0.9639
INFO:root:Iter 3: Q = 74666.8515625, term1 = 117331.796875, term2 = 32529.8125
 20%|██        | 3/15 [00:03<00:12,  1.02s/it]INFO:root:U_1-Qu_upper=-147.3650455819349, U_2-Qv_upper=-77.6301257036552
INFO:root:eta=0.5146562042904931, l=0.278, r=0.929605
INFO:root:Iter 4: Q = 12691442.0, term

In [55]:
FACTUAL,COUNTERFACTUAL = discount_test_scaler.generate_counterfactuals(
    data=compas_ares_scaler_test,
    lr=1e-1,
    n_proj=10,
    delta=0.05,
    U_1=0.4,
    U_2=0.3,
    l=0.2,
    r=1,
    max_iter=15,
    tau=1e2,   #差值变化 - 步长不能太大或者太小
    silent=False,
)

In [57]:
FACTUAL

array([[-0.48239933,  0.48239933, -0.52162709,  0.85929057, -0.51691114,
        -1.03820131, -0.07552357,  1.41206789, -0.30054426, -0.04028706,
        -0.24613009, -1.35248895,  1.35248895, -0.05857799, -0.24907213],
       [-0.48239933,  0.48239933, -0.52162709,  0.85929057, -0.51691114,
        -1.03820131, -0.07552357, -0.70818125,  3.3272969 , -0.04028706,
        -0.24613009, -1.35248895,  1.35248895, -0.26998115, -0.31448817],
       [-0.48239933,  0.48239933, -0.52162709,  0.85929057, -0.51691114,
        -1.03820131, -0.07552357,  1.41206789, -0.30054426, -0.04028706,
        -0.24613009, -1.35248895,  1.35248895, -0.05857799, -0.11824003],
       [-0.48239933,  0.48239933, -0.52162709,  0.85929057, -0.51691114,
        -1.03820131, -0.07552357,  1.41206789, -0.30054426, -0.04028706,
        -0.24613009, -1.35248895,  1.35248895, -0.69278746,  0.09981347]])

In [58]:
COUNTERFACTUAL

array([[ 7.60768632e+19, -2.43854109e+19, -1.01902936e+19,
         1.70677168e+19, -1.07778581e+20,  2.39597284e+19,
        -4.28363441e+19, -7.02984370e+19,  6.71706002e+18,
        -8.49078239e+19,  6.11193445e+19, -4.84563878e+19,
        -1.46901025e+20,  1.82407671e+19,  3.46833972e+19],
       [ 1.51145817e+20, -5.84598986e+19, -3.55969194e+20,
        -8.29703437e+19, -9.97626658e+19, -1.38731473e+19,
         2.25533532e+20, -2.98642311e+19, -1.08800018e+20,
        -8.72218825e+18, -1.39998520e+20, -3.07962722e+20,
        -1.41065741e+20,  2.90366296e+20, -5.22937318e+19],
       [ 6.58518774e+16,  5.37129847e+17, -5.43214819e+17,
        -3.98472154e+17, -3.79203487e+17,  5.74257572e+17,
        -1.10609702e+18, -3.79699590e+15,  1.01860633e+18,
        -1.06010115e+18, -9.68061749e+16, -1.01201112e+18,
        -8.48801880e+17,  1.09700670e+16, -8.94914985e+17],
       [ 2.21090431e+20, -1.55575354e+20, -3.31973769e+20,
         1.30074163e+18, -8.29704756e+19, -3.98146091

In [59]:
inversed = scaler.inverse_transform(COUNTERFACTUAL)
inversed

array([[ 2.9771360e+19, -9.5428077e+18, -4.1785667e+18,  8.4366627e+18,
        -4.3964707e+19,  1.1971451e+19, -3.2168054e+18, -3.3155745e+19,
         1.8515309e+18, -3.4151439e+18,  1.4184041e+19, -2.3164188e+19,
        -7.0224858e+19,  8.6284272e+19,  1.5905912e+21],
       [ 5.9148294e+19, -2.2877268e+19, -1.4596646e+20, -4.1012681e+19,
        -4.0694878e+19, -6.9317017e+18,  1.6936494e+19, -1.4085246e+19,
        -2.9990292e+19, -3.5082193e+17, -3.2489625e+19, -1.4721911e+20,
        -6.7435344e+19,  1.3735192e+21, -2.3982065e+21],
       [ 2.5769991e+16,  2.1019646e+17, -2.2274721e+17, -1.9696689e+17,
        -1.5468352e+17,  2.8692713e+17, -8.3062623e+16, -1.7908254e+15,
         2.8077479e+17, -4.2639155e+16, -2.2465926e+16, -4.8378378e+17,
        -4.0576291e+17,  5.1891687e+16, -4.1041075e+19],
       [ 8.6519901e+19, -6.0881718e+19, -1.3612704e+20,  6.4296348e+17,
        -3.3845059e+19, -1.9893323e+19,  2.2050103e+19,  2.7577885e+19,
        -5.9864877e+19,  2.9658323e+1